Players dataset creation

The Fantacalcio players list is manually downloaded from https://www.fantacalcio.it/quotazioni-fantacalcio

Here, the players database is generated, by merging the Fantacalcio list to stats downloaded from http://fbref.com

In [50]:
import pandas as pd

Load fbref data for outfield players and goalkeepers.

Generate fbref player list, adding player surname (with special characters replaced to normal ones)

In [51]:
rcsv = pd.read_csv('fbref_data/outfield_players.csv')   
outfield_players = pd.DataFrame(rcsv)

rcsv = pd.read_csv('fbref_data/keepers_players.csv')   
keeper_players = pd.DataFrame(rcsv)

In [52]:
players = pd.concat(   [ outfield_players[['player', 'team']], keeper_players[['player', 'team']] ], axis = 0, ignore_index = True)

keepers_ID = len(outfield_players)

players

,player,team
0,Francesco Acerbi,Inter
1,Yacine Adli,Milan
2,Michel Aebischer,Bologna
3,Lucien Agoume,Inter
4,Marley Aké,Udinese
...,...,...
530,Alessandro Sorrentino,Monza
531,Marco Sportiello,Milan
532,Wojciech Szczęsny,Juventus
533,Pietro Terracciano,Fiorentina


In [53]:
import unicodedata

def normalize_name(input_str):
    nfkd_form = unicodedata.normalize('NFKD', input_str)
    only_ascii = nfkd_form.encode('ASCII', 'ignore')
    return only_ascii.decode('utf-8')

In [54]:
players['surname'] = players['player']
players['initial'] = players['player']

for i in range(players.shape[0]):
    players['surname'][i] = players['surname'][i].split(' ')[-1]
    players['surname'][i] = normalize_name(players['surname'][i]).replace('\'', '')
    
    
    players['initial'][i] = players['player'][i][0]

In [55]:
print(players[['player', 'surname']].to_string())

                         player           surname
0              Francesco Acerbi            Acerbi
1                   Yacine Adli              Adli
2              Michel Aebischer         Aebischer
3                 Lucien Agoume            Agoume
4                    Marley Aké               Ake
5        Jean-Daniel Akpa-Akpro        Akpa-Akpro
6                  Luis Alberto           Alberto
7               Pontus Almqvist          Almqvist
8              Lorenzo Amatucci          Amatucci
9                  Bruno Amione            Amione
10              Felipe Anderson          Anderson
11                Houssem Aouar             Aouar
12             Marko Arnautović        Arnautovic
13             Kristjan Asllani           Asllani
14              Tommaso Augello           Augello
15           Yann Aurel Bisseck           Bisseck
16                Sardar Azmoun            Azmoun
17                   Paulo Azzi              Azzi
18           Oussama El Azzouzi           Azzouzi


Replace the surname for some specific players, according to config/name_fix.txt file.

This is done for players for which the decoded fbref surname doesn't correspond to Fantacalcio list.

In [56]:
rcsv = pd.read_csv('config/name_fix.txt')   
name_fix = pd.DataFrame(rcsv)

for i in range(name_fix.shape[0]):
    for j in range(players.shape[0]):
        if(players['surname'][j].lower() == name_fix['FROM'][i].lower() and players['team'][j].lower() == name_fix['TEAM'][i].lower()):
            players['surname'][j] = name_fix['TO'][i]
            print(name_fix['TO'][i])

name_fix



Ostigard
Augusto
Kjaer
Djuric
Gudmundsson


,FROM,TO,TEAM
0,stigard,Ostigard,Napoli
1,Min-jae,Kim,Napoli
2,Hjlund,Hojlund,Atalanta
3,Gytkjr,Gytkjaer,Monza
4,Carlos,Augusto,Inter
5,Mhle,Maehle,Atalanta
6,Kjr,Kjaer,Milan
7,uricic,Djuricic,Sampdoria
8,uric,Djuric,Hellas Verona
9,Arthur,Cabral,Fiorentina


Load players from Fantacalcio list.

In [57]:
fc_data = pd.read_excel('fantacalcio/Quotazioni_Fantacalcio.xlsx', 'Tutti', header = 1)

fc_players = fc_data [['Id', 'R', 'Nome', 'Squadra']]

fc_players = fc_players.rename(columns = {'Id' : 'id', 'R': 'r', 'Nome' : 'name', 'Squadra' : 'team'})

fc_players['surname'] = fc_players['name']
fc_players['initial'] = fc_players['name']


for i in range(fc_players.shape[0]):
    spl = normalize_name( fc_players['name'][i].replace('\'', '') ).split(' ')
    if('.' in spl[-1]):
        fc_players.loc[i, 'surname'] = spl[-2]
        fc_players.loc[i, 'initial'] = spl[-1][0]
    else:
        fc_players.loc[i, 'surname'] = spl[-1]
        fc_players.loc[i, 'initial'] = ''
    
fc_players



,id,r,name,team,surname,initial
0,2428,P,Sommer,Inter,Sommer,
1,133,P,Skorupski,Bologna,Skorupski,
2,5876,P,Di Gregorio,Monza,Gregorio,
3,572,P,Meret,Napoli,Meret,
4,2814,P,Provedel,Lazio,Provedel,
...,...,...,...,...,...,...
537,6160,A,Vivaldo,Udinese,Vivaldo,
538,6242,A,Bidaoui,Frosinone,Bidaoui,
539,6418,A,Burnete,Lecce,Burnete,
540,6419,A,Corfitzen,Lecce,Corfitzen,


Associate players from Fantacalcio list to ID for FBref data.

In [77]:
fc_players['fb_ID'] = fc_players['id']

for i in range(fc_players.shape[0]):
    fc_players.loc[i, 'fb_ID']  = -1
    
    for j in range(players.shape[0]):
        if(fc_players['team'][i].lower() in players['team'][j].lower()):
            if(fc_players['surname'][i].lower() == players['surname'][j].lower()):              
                # if(fc_players['initial'][i] == '' or fc_players['initial'][i].lower() == players['initial'][j].lower()):
                if((fc_players['r'][i] == 'P') == (j >= keepers_ID)): # check wether they're a goalkeeper for both FBREF and Fantacalcio
                    fc_players.loc[i, 'fb_ID'] = j

fc_players          
        

,id,r,name,team,surname,initial,fb_ID,age,birth_year,games,...,gk_pct_goal_kicks_launched,gk_goal_kick_length_avg,gk_crosses,gk_crosses_stopped,gk_crosses_stopped_pct,gk_def_actions_outside_pen_area,gk_def_actions_outside_pen_area_per90,gk_avg_distance_def_actions,vote_avg,vote_std
0,2428,P,Sommer,Inter,Sommer,,529,34-354,1988,0,...,30.0,30.3,149,7,4.7,8,0.57,12.1,5.857143,0.349927
1,133,P,Skorupski,Bologna,Skorupski,,528,32-215,1991,0,...,41.5,38.7,205,6,2.9,3,0.21,9.0,6.357143,0.349927
2,5876,P,Di Gregorio,Monza,Gregorio,,508,26-132,1997,0,...,45.6,42.2,194,9,4.6,6,0.46,10.2,6.583333,0.533594
3,572,P,Meret,Napoli,Meret,,515,26-259,1997,0,...,25.0,30.1,143,1,0.7,12,1.00,16.8,5.857143,0.225877
4,2814,P,Provedel,Lazio,Provedel,,524,29-264,1994,0,...,25.4,30.7,152,6,3.9,10,0.71,13.4,6.428571,0.416497
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
537,6160,A,Vivaldo,Udinese,Vivaldo,,-1,0,0,0,...,0.0,0.0,0,0,0.0,0,0.00,0.0,6.172900,0.297326
538,6242,A,Bidaoui,Frosinone,Bidaoui,,-1,0,0,0,...,0.0,0.0,0,0,0.0,0,0.00,0.0,6.036372,0.496942
539,6418,A,Burnete,Lecce,Burnete,,65,19-309,2004,1,...,0.0,0.0,0,0,0.0,0,0.00,0.0,6.292575,0.210428
540,6419,A,Corfitzen,Lecce,Corfitzen,,106,18-342,2004,1,...,0.0,0.0,0,0,0.0,0,0.00,0.0,6.192570,0.614719


Print players for which the association failed.

Most of them are players who didn't play a single Serie A game this season with their team. If that is the case, and there is data from their previous team, that is taken here.

Others are ones for which the FBRef surname doesn't correspond to Fantacalcio one.


For example, Cabral is Arthur for FBref.

Correction is made in the name_fix code above.

In [59]:
exceptions = ['pellegrini', 'bastoni', 'kristensen'] # exceptions for such players that have the same surname as others (Berardi A., Luca Pellegrini)

for i in range(fc_players.shape[0]):
    if(fc_players['fb_ID'][i] == -1):
        found = False
        for j in range(players.shape[0]):
            if(fc_players['surname'][i].lower() == players['surname'][j].lower()):
                if(not(players['surname'][j].lower() in exceptions)):
                    if((fc_players['r'][i] == 'P') == (j >= keepers_ID)):
                        fc_players.loc[i, 'fb_ID']= j
                        found = True
        if(found):
            print(fc_players['name'][i] + ' from previous team stats')
        else:
            print(fc_players['name'][i] + ' not found')

Sepe not found
Lamanna not found
Sommariva not found
Pegolo not found
Perilli not found
Padelli not found
Audero not found
Di Gennaro not found
Pinsoglio not found
Aresti not found
Fiorillo not found
Rossi F. not found
Ravaglia F. not found
Frattali not found
Contini not found
Brancolini not found
Berardi A. not found
Boer not found
Bagnolini not found
Svilar not found
Martinelli T. not found
Popa not found
Stubljar not found
Gori not found
Borbei not found
Okoye not found
Mandas not found
Kristensen not found
Masina not found
Tressoldi not found
Gunter not found
Soumaoro not found
Dermaku not found
Tonelli not found
De Sciglio not found
Capradossi not found
Bettella not found
Kumbulla not found
Amey not found
Cittadini not found
Guarino not found
Smajlovic not found
N'guessan not found
Mateus Lusuardi not found
Kalaj not found
Pierozzi not found
Lindstrom not found
Castrovilli not found
Iling Junior not found
Cajuste  not found
Machin not found
Basic not found
Legowski not found
Lulic

In [60]:
fc_players

,id,r,name,team,surname,initial,fb_ID
0,2428,P,Sommer,Inter,Sommer,,529
1,133,P,Skorupski,Bologna,Skorupski,,528
2,5876,P,Di Gregorio,Monza,Gregorio,,508
3,572,P,Meret,Napoli,Meret,,515
4,2814,P,Provedel,Lazio,Provedel,,524
...,...,...,...,...,...,...,...
537,6160,A,Vivaldo,Udinese,Vivaldo,,-1
538,6242,A,Bidaoui,Frosinone,Bidaoui,,-1
539,6418,A,Burnete,Lecce,Burnete,,65
540,6419,A,Corfitzen,Lecce,Corfitzen,,106


Populate players dataset with stats from FBref, for outfield players and goalkeepers

In [61]:
#Data for Outfield players
columns_to_copy = outfield_players.columns[4:]

fc_players[columns_to_copy] = 0

for i in range(fc_players.shape[0]):
    if(fc_players['fb_ID'][i] != -1 and fc_players['r'][i] != 'P'):
         for j in range(columns_to_copy.shape[0]):
            #fc_players[columns_to_copy[j]][i] = outfield_players[columns_to_copy[j]][fc_players['fb_ID'][i]]
            fc_players.loc[i, columns_to_copy[j]] = outfield_players.loc[fc_players['fb_ID'][i], columns_to_copy[j]]
            

/tmp/ipykernel_33524/2053436513.py:4: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  fc_players[columns_to_copy] = 0
/tmp/ipykernel_33524/2053436513.py:4: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  fc_players[columns_to_copy] = 0
/tmp/ipykernel_33524/2053436513.py:4: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = f

In [62]:
keeper_players.columns[4:]

Index(['age', 'birth_year', 'gk_games', 'gk_games_starts', 'gk_minutes',
       'gk_goals_against', 'gk_goals_against_per90',
       'gk_shots_on_target_against', 'gk_saves', 'gk_save_pct', 'gk_wins',
       'gk_ties', 'gk_losses', 'gk_clean_sheets', 'gk_clean_sheets_pct',
       'gk_pens_att', 'gk_pens_allowed', 'gk_pens_saved', 'gk_pens_missed',
       'minutes_90s', 'gk_free_kick_goals_against',
       'gk_corner_kick_goals_against', 'gk_own_goals_against', 'gk_psxg',
       'gk_psnpxg_per_shot_on_target_against', 'gk_psxg_net',
       'gk_psxg_net_per90', 'gk_passes_completed_launched',
       'gk_passes_launched', 'gk_passes_pct_launched', 'gk_passes',
       'gk_passes_throws', 'gk_pct_passes_launched', 'gk_passes_length_avg',
       'gk_goal_kicks', 'gk_pct_goal_kicks_launched',
       'gk_goal_kick_length_avg', 'gk_crosses', 'gk_crosses_stopped',
       'gk_crosses_stopped_pct', 'gk_def_actions_outside_pen_area',
       'gk_def_actions_outside_pen_area_per90', 'gk_avg_distance_

In [63]:
#Data for Keepers
columns_to_copy = keeper_players.columns[4:]

fc_players[columns_to_copy[2:]] = 0 # add columns but do not override age and birth_year

delta_k = outfield_players.shape[0]

for i in range(fc_players.shape[0]):
    if(fc_players['fb_ID'][i] != -1 and fc_players['r'][i] == 'P'):
         for j in range(columns_to_copy.shape[0]):
            #fc_players[columns_to_copy[j]][i] = keeper_players[columns_to_copy[j]][fc_players['fb_ID'][i] - delta_k]
            fc_players.loc[i, columns_to_copy[j]] = keeper_players.loc[fc_players['fb_ID'][i] - delta_k, columns_to_copy[j]]

/tmp/ipykernel_33524/127446819.py:4: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  fc_players[columns_to_copy[2:]] = 0 # add columns but do not override age and birth_year
/tmp/ipykernel_33524/127446819.py:4: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  fc_players[columns_to_copy[2:]] = 0 # add columns but do not override age and birth_year
/tmp/ipykernel_33524/127446819.py:4: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consid

In [64]:
fc_players

,id,r,name,team,surname,initial,fb_ID,age,birth_year,games,...,gk_passes_length_avg,gk_goal_kicks,gk_pct_goal_kicks_launched,gk_goal_kick_length_avg,gk_crosses,gk_crosses_stopped,gk_crosses_stopped_pct,gk_def_actions_outside_pen_area,gk_def_actions_outside_pen_area_per90,gk_avg_distance_def_actions
0,2428,P,Sommer,Inter,Sommer,,529,34-354,1988,0,...,28.3,80,30.0,30.3,149,7,4.7,8,0.57,12.1
1,133,P,Skorupski,Bologna,Skorupski,,528,32-215,1991,0,...,30.2,82,41.5,38.7,205,6,2.9,3,0.21,9.0
2,5876,P,Di Gregorio,Monza,Gregorio,,508,26-132,1997,0,...,29.3,90,45.6,42.2,194,9,4.6,6,0.46,10.2
3,572,P,Meret,Napoli,Meret,,515,26-259,1997,0,...,27.3,56,25.0,30.1,143,1,0.7,12,1.00,16.8
4,2814,P,Provedel,Lazio,Provedel,,524,29-264,1994,0,...,28.4,63,25.4,30.7,152,6,3.9,10,0.71,13.4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
537,6160,A,Vivaldo,Udinese,Vivaldo,,-1,0,0,0,...,0.0,0,0.0,0.0,0,0,0.0,0,0.00,0.0
538,6242,A,Bidaoui,Frosinone,Bidaoui,,-1,0,0,0,...,0.0,0,0.0,0.0,0,0,0.0,0,0.00,0.0
539,6418,A,Burnete,Lecce,Burnete,,65,19-309,2004,1,...,0.0,0,0.0,0.0,0,0,0.0,0,0.00,0.0
540,6419,A,Corfitzen,Lecce,Corfitzen,,106,18-342,2004,1,...,0.0,0,0.0,0.0,0,0,0.0,0,0.00,0.0


Load votes database, to add data to players database (mean vote and its standard deviation)

In [65]:
import numpy as np

votes = pd.read_excel('mid_outputs/players_votes.xlsx', index_col = 0)

Compute the average Serie A Goal Keeper mean vote and vote std

In [66]:
min_votes = 3 # TO BE UPDATED WHEN SERIE A HAS MORE CALENDAR WEEKS PLAYED

perf_df_P = pd.DataFrame(columns = ['vote_avg', 'vote_std'])

for i in range(fc_players.shape[0]): 
    if(fc_players.loc[i]['r'] == 'P'):
        v = np.array([])
        for j in range(votes.shape[0]):
            if(fc_players['name'][i] == votes['player'][j]):
                v = np.append(v, votes['vote'][j])

        if(v.shape[0] >= min_votes - 1):
            row_df = pd.DataFrame(data = [[np.mean(v), np.std(v)]], columns = perf_df_P.columns)
            perf_df_P = pd.concat([perf_df_P, row_df], ignore_index = True)


print(perf_df_P.mean())

perf_df_P


/tmp/ipykernel_33524/3315699811.py:14: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  perf_df_P = pd.concat([perf_df_P, row_df], ignore_index = True)


vote_avg    6.102000
vote_std    0.378953
dtype: float64


,vote_avg,vote_std
0,5.857143,0.349927
1,6.357143,0.349927
2,6.583333,0.533594
3,5.857143,0.225877
4,6.428571,0.416497
5,6.200000,0.678233
6,6.000000,1.048809
7,6.357143,0.440315
8,6.000000,0.267261
9,6.000000,0.000000


Add to players data their mean vote (and its standard deviation).

For players who don't have a minimum amount of games, more data to reach this value is computed, according to the average Serie A player vote (and std). For goalkeepers, this values are different.


In [67]:
min_votes = 3 # TO BE UPDATED WHEN SERIE A HAS MORE CALENDAR WEEKS PLAYED

#outfield players
mean_def = 6
std_def = 0.58

#goalkeepers
mean_def_P = 6.22
std_def_P = 0.43


perf_df = pd.DataFrame(columns = ['vote_avg', 'vote_std'])

for i in range(fc_players.shape[0]):
    v = np.array([])
    for j in range(votes.shape[0]):
        if(fc_players['name'][i] == votes['player'][j]):
            v = np.append(v, votes['vote'][j])
    
    mean_def_i = mean_def
    std_def_i = std_def
    
    if(fc_players['r'][i] == 'P'):
        mean_def_i = mean_def_P
        std_def_i = std_def_P
        
    if(v.shape[0] < min_votes):
         for k in range(min_votes - v.shape[0]):
            v = np.append( v, np.random.normal(mean_def_i, std_def_i) )
    
    row_df = pd.DataFrame(data = [[np.mean(v), np.std(v)]], columns = perf_df.columns)
    perf_df = pd.concat([perf_df, row_df], ignore_index = True)
    
perf_df
    

/tmp/ipykernel_33524/1006814721.py:32: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  perf_df = pd.concat([perf_df, row_df], ignore_index = True)


,vote_avg,vote_std
0,5.857143,0.349927
1,6.357143,0.349927
2,6.583333,0.533594
3,5.857143,0.225877
4,6.428571,0.416497
...,...,...
537,6.172900,0.297326
538,6.036372,0.496942
539,6.292575,0.210428
540,6.192570,0.614719


In [68]:
fc_players = pd.concat([fc_players, perf_df], axis = 1)

fc_players

,id,r,name,team,surname,initial,fb_ID,age,birth_year,games,...,gk_pct_goal_kicks_launched,gk_goal_kick_length_avg,gk_crosses,gk_crosses_stopped,gk_crosses_stopped_pct,gk_def_actions_outside_pen_area,gk_def_actions_outside_pen_area_per90,gk_avg_distance_def_actions,vote_avg,vote_std
0,2428,P,Sommer,Inter,Sommer,,529,34-354,1988,0,...,30.0,30.3,149,7,4.7,8,0.57,12.1,5.857143,0.349927
1,133,P,Skorupski,Bologna,Skorupski,,528,32-215,1991,0,...,41.5,38.7,205,6,2.9,3,0.21,9.0,6.357143,0.349927
2,5876,P,Di Gregorio,Monza,Gregorio,,508,26-132,1997,0,...,45.6,42.2,194,9,4.6,6,0.46,10.2,6.583333,0.533594
3,572,P,Meret,Napoli,Meret,,515,26-259,1997,0,...,25.0,30.1,143,1,0.7,12,1.00,16.8,5.857143,0.225877
4,2814,P,Provedel,Lazio,Provedel,,524,29-264,1994,0,...,25.4,30.7,152,6,3.9,10,0.71,13.4,6.428571,0.416497
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
537,6160,A,Vivaldo,Udinese,Vivaldo,,-1,0,0,0,...,0.0,0.0,0,0,0.0,0,0.00,0.0,6.172900,0.297326
538,6242,A,Bidaoui,Frosinone,Bidaoui,,-1,0,0,0,...,0.0,0.0,0,0,0.0,0,0.00,0.0,6.036372,0.496942
539,6418,A,Burnete,Lecce,Burnete,,65,19-309,2004,1,...,0.0,0.0,0,0,0.0,0,0.00,0.0,6.292575,0.210428
540,6419,A,Corfitzen,Lecce,Corfitzen,,106,18-342,2004,1,...,0.0,0.0,0,0,0.0,0,0.00,0.0,6.192570,0.614719


In [69]:
GK_GAMES_COLUMN = 118

fc_players.columns[GK_GAMES_COLUMN]

# check it if is 'gk_games'

'gk_games'

For goalkeepers who didn't play a miminum amount of games, data is weightly averaged with one of the main goalkeeper of their same team.

In [70]:
min_gk_games = 3 # TO BE UPDATED WHEN SERIE A HAS MORE CALENDAR WEEKS PLAYED

fc_players_newgk = fc_players.copy()

columns_to_avg = fc_players.columns[GK_GAMES_COLUMN:] # from gk_games to end

for i in range(fc_players.shape[0]):
    if(fc_players['r'][i] == 'P'):
        if(fc_players['gk_games'][i] < min_gk_games):
            for j in range(fc_players.shape[0]):
                if(fc_players['team'][i] == fc_players['team'][j] and fc_players['gk_games'][j] >= min_gk_games):
                    break
                    
            weight = 1 - (min_gk_games - fc_players['gk_games'][i]) / min_gk_games
            
            fc_players_newgk.at[i, columns_to_avg] = fc_players.loc[i][columns_to_avg] * weight + (1 - weight) * fc_players.loc[j][columns_to_avg]
            
            print(fc_players['name'][i] + ', ' + str(weight))
            
        

InvalidIndexError: You can only assign a scalar value not a <class 'pandas.core.series.Series'>

In [71]:
fc_players = fc_players_newgk

fc_players

,id,r,name,team,surname,initial,fb_ID,age,birth_year,games,...,gk_pct_goal_kicks_launched,gk_goal_kick_length_avg,gk_crosses,gk_crosses_stopped,gk_crosses_stopped_pct,gk_def_actions_outside_pen_area,gk_def_actions_outside_pen_area_per90,gk_avg_distance_def_actions,vote_avg,vote_std
0,2428,P,Sommer,Inter,Sommer,,529,34-354,1988,0,...,30.0,30.3,149,7,4.7,8,0.57,12.1,5.857143,0.349927
1,133,P,Skorupski,Bologna,Skorupski,,528,32-215,1991,0,...,41.5,38.7,205,6,2.9,3,0.21,9.0,6.357143,0.349927
2,5876,P,Di Gregorio,Monza,Gregorio,,508,26-132,1997,0,...,45.6,42.2,194,9,4.6,6,0.46,10.2,6.583333,0.533594
3,572,P,Meret,Napoli,Meret,,515,26-259,1997,0,...,25.0,30.1,143,1,0.7,12,1.00,16.8,5.857143,0.225877
4,2814,P,Provedel,Lazio,Provedel,,524,29-264,1994,0,...,25.4,30.7,152,6,3.9,10,0.71,13.4,6.428571,0.416497
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
537,6160,A,Vivaldo,Udinese,Vivaldo,,-1,0,0,0,...,0.0,0.0,0,0,0.0,0,0.00,0.0,6.172900,0.297326
538,6242,A,Bidaoui,Frosinone,Bidaoui,,-1,0,0,0,...,0.0,0.0,0,0,0.0,0,0.00,0.0,6.036372,0.496942
539,6418,A,Burnete,Lecce,Burnete,,65,19-309,2004,1,...,0.0,0.0,0,0,0.0,0,0.00,0.0,6.292575,0.210428
540,6419,A,Corfitzen,Lecce,Corfitzen,,106,18-342,2004,1,...,0.0,0.0,0,0,0.0,0,0.00,0.0,6.192570,0.614719


Save to file.

In [72]:
fc_players.to_excel('mid_outputs/players_stats.xlsx')